### 0. Import libraries

In [15]:
import pandas as pd
from datetime import datetime, timedelta
import os
import numpy as np
from natsort import index_natsorted, order_by_index
import math

### 1. Import data

In [16]:
df_jams = pd.read_csv('../dataset/jams-raw.csv')
df_jam_desc = pd.read_csv('../dataset/jam_desc.csv')

### 2. Clean data and preprocess features

In [ ]:
# Fill jams with no ratings information with 0
df_jams['jam_no_rating'].fillna('0', inplace = True)

# Remove ',' from the number
df_jams['jam_no_rating'] = df_jams['jam_no_rating'].str.replace(',', '')

# Convert k, m to 000 and 000000
df_jams['jam_no_rating'] = (df_jams['jam_no_rating'].replace(r'[km]+$', '', regex=True).astype(float) * \
                            df_jams['jam_no_rating'].str.extract(r'[\d\.]+([km]+)', expand=False)
                                                    .fillna(1)
                                                    .replace(['k','m'], [10**3, 10**6]).astype(int)).astype(int)

In [17]:
# get jams with duration > 1 hour and truncate top 1% jams
date_pattern = "%Y-%m-%d %H:%M:%S" # 2016-11-14 12:00:00
df_jams["jam_duration"] = (pd.to_datetime(df_jams['jam_end_date'], format=date_pattern) - \
                           pd.to_datetime(df_jams['jam_start_date'], format=date_pattern)) / \
                           timedelta(hours = 1)
df_jams = df_jams[df_jams['jam_duration'] > 1] # only get jams lasting more than 1 hour
df_jams.sort_values(by=['jam_duration'], 
                    ascending=True, 
                    inplace=True)
df_jams = df_jams.head(int(len(df_jams)*(99/100))) # remove top 1% in length

In [18]:
# join jam and jam description
df_jam_all = pd.merge(df_jams, df_jam_desc, on='jam_url', how='inner')

In [19]:
# Calculate number of hosts
df_jam_all["num_hosts"] = df_jam_all["jam_host"].map(lambda a: len(a.split("||")))

In [20]:
# Separate into competitive jam and non-competitive jam
competitive_jams = df_jam_all[df_jam_all['jam_criteria'].notnull()]
non_competitive_jams = df_jam_all[df_jam_all['jam_criteria'].isnull()]

In [21]:
# Calculate number of criteria for competitive jam
competitive_jams["num_criteria"] = competitive_jams["jam_criteria"].map(lambda a: len(a.split("||")))

### 3. Separate jams into top 20% and bottom 20% in terms of number of submissions (for binary logistic regresssion)

In [22]:
# Get top and bottom 20% jam by popularity (number of submissions) - Competitive jams
top_n = math.ceil(len(competitive_jams)*(20/100))
competitive_jams = competitive_jams.reindex(index=order_by_index(competitive_jams.index,
                                                                index_natsorted(competitive_jams['jam_no_submissions'],
                                            reverse=True)))
top_20_jam = competitive_jams.head(top_n)
top_20_jam.insert(len(top_20_jam.columns), 
                    'popular',
                    pd.Series("Yes", index=top_20_jam.index))
# frames.append(top_20_jam)

bottom_20_jam = competitive_jams.tail(top_n)
bottom_20_jam.insert(len(bottom_20_jam.columns), 
                        'popular',
                        pd.Series("No", index=bottom_20_jam.index))
# frames.append(bottom_20_jam)
final_competitive_jams = pd.concat([top_20_jam, bottom_20_jam])

In [23]:
# Get top and bottom 20% jam by popularity (number of submissions) - Non-Competitive jams
top_n = math.ceil(len(non_competitive_jams)*(20/100))
non_competitive_jams = non_competitive_jams.reindex(index=order_by_index(non_competitive_jams.index,
                                                                        index_natsorted(non_competitive_jams['jam_no_submissions'],
                                                    reverse=True)))
top_20_jam = non_competitive_jams.head(top_n)
top_20_jam.insert(len(top_20_jam.columns), 
                    'popular',
                    pd.Series("Yes", index=top_20_jam.index))
# frames.append(top_20_jam)

bottom_20_jam = non_competitive_jams.tail(top_n)
bottom_20_jam.insert(len(bottom_20_jam.columns), 
                        'popular',
                        pd.Series("No", index=bottom_20_jam.index))
# frames.append(bottom_20_jam)
final_non_competitive_jams = pd.concat([top_20_jam, bottom_20_jam])

### 4. Writing to csv

In [24]:
# Write competitive_jams dataset
output_file = "../dataset/competitive_jams_cleaned.csv"
if os.path.exists(output_file):
    os.remove(output_file)
final_competitive_jams.to_csv(output_file, encoding='utf-8-sig', index=False)

In [25]:
# Write non competitive_jams dataset
output_file = "../dataset/non_competitive_jams_cleaned.csv"
if os.path.exists(output_file):
    os.remove(output_file)
final_non_competitive_jams.to_csv(output_file, encoding='utf-8-sig', index=False)